In [1]:
import json
set_dir = 'test'
f = open('{}.json'.format(set_dir))

In [2]:
dat_json = json.load(f)
f.close()
len(dat_json)

1000

In [3]:
import soundfile as sf
import librosa
import torch
# data, sr = sf.read('1.wav')
# data = librosa.resample(data[:,0],sr,16000)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)
from transformers import Wav2Vec2FeatureExtractor, Wav2Vec2Model
import torch

model_name = "facebook/wav2vec2-large-xlsr-53"
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(model_name)
model_audio = Wav2Vec2Model.from_pretrained(model_name).to(device)

# i= feature_extractor(data, return_tensors="pt", sampling_rate=16000)
# #previous are in dataloader

# with torch.no_grad():
#   o= model_audio(i.input_values)

/usr2/home/haozhez2/anaconda3/envs/asr/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Device:  cuda


Some weights of the model checkpoint at facebook/wav2vec2-large-xlsr-53 were not used when initializing Wav2Vec2Model: ['project_hid.weight', 'project_q.weight', 'quantizer.weight_proj.weight', 'project_q.bias', 'quantizer.weight_proj.bias', 'quantizer.codevectors', 'project_hid.bias']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
import numpy as np
from tqdm import tqdm

file_path = set_dir+'/'
idx = 0
dat_combined_json = []
for dat in tqdm(dat_json):
    text = dat['description']
    audio_list = dat['audio_ids']
    tar_id = dat['target_id']
    final_feat = []
    final_target = []
    final_hidden_state = []
    for audio_name in audio_list:
        tmp_audio, sr = sf.read(file_path+audio_name+'.wav')
        tmp_audio = librosa.resample(tmp_audio[:,0],sr,16000)
        i= feature_extractor(tmp_audio, return_tensors="pt", sampling_rate=16000)

        with torch.no_grad():
            o= model_audio(i.input_values.to(device))

        cur_feat = o.extract_features.cpu().numpy()
        cur_hidden_state = o.last_hidden_state.cpu().numpy()
        B, T, D = cur_feat.shape
        # print(B,T,D)


        tmp_tar = np.zeros(T)
        if audio_name == tar_id:
            tmp_tar += 1

        final_feat.append(np.squeeze(cur_feat))
        final_hidden_state.append(np.squeeze(cur_hidden_state))
        final_target.append(tmp_tar)
    tmp_dict = {}
    tmp_dict['id'] = idx
    tmp_dict['audio_ids'] = audio_list
    tmp_dict['target_id'] = tar_id
    tmp_dict['description'] = text
    final_feat = np.vstack(final_feat)
    final_hidden_state = np.vstack(final_hidden_state)
    final_target = np.hstack(final_target)
    # print(final_feat.shape, final_hidden_state.shape, final_target.shape)
    np.save('./'+set_dir+'/'+str(idx)+'_feat', final_feat)
    np.save('./'+set_dir+'/'+str(idx)+'_hidden_state', final_hidden_state)
    np.save('./'+set_dir+'/'+str(idx)+'_target', final_target)
    dat_combined_json.append(tmp_dict)

    idx += 1




  0%|          | 0/1000 [00:00<?, ?it/s]/tmp/ipykernel_441495/3862239507.py:16: FutureWarning: Pass orig_sr=48000, target_sr=16000 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  tmp_audio = librosa.resample(tmp_audio[:,0],sr,16000)
/tmp/ipykernel_441495/3862239507.py:16: FutureWarning: Pass orig_sr=48000, target_sr=16000 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  tmp_audio = librosa.resample(tmp_audio[:,0],sr,16000)
100%|██████████| 1000/1000 [07:30<00:00,  2.22it/s]


In [6]:
with open('{}_combined.json'.format(set_dir), "w") as outfile:
    json.dump(dat_combined_json, outfile)

In [7]:
len(dat_combined_json)

1000